<a href="https://colab.research.google.com/github/LeonSilva15/fcc-machine-learning/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import unittest
import random

## Context

In [174]:
actions = ['R', 'P', 'S']
counter_play = {
    'R': 'P',
    'P': 'S',
    'S': 'R'
}
probabilities = {}
look_back_size = 5

## My machine learning player

In [175]:
def player(prev_play, opponent_history=[]):
  if prev_play:
    opponent_history.append(prev_play)

  if len(opponent_history) > look_back_size:
    # Create the last plays chunk as a single string key
    last_play = ''.join( opponent_history[-look_back_size:] )
    # Index to verify if this chunk previously existed
    existing_last_play = ''.join( opponent_history[-(look_back_size + 1):] )

    # Look if this is an existing pattern and increase its probability
    if existing_last_play in probabilities:
      probabilities[existing_last_play] += 1
    # If it doesn't exist, add it
    else:
      probabilities[existing_last_play] = 1

    # Add the candidates in 0 so we can use them in the prediciton
    candidates = [last_play + 'R', last_play + 'P', last_play + 'S']
    for candidate in candidates:
      if not candidate in probabilities.keys():
        probabilities[candidate] = 0

    # Get the candidate with the most probabilities
    prediction = max(candidates, key=lambda key: probabilities[key])[-1]

    # Return the counter play of the prediction
    return counter_play[ prediction ]

  return random.choice(actions)

## Game creation

In [176]:
def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])

## Automated playing

In [177]:
play(player, quincy, 1000)
play(player, abbey, 1000)
play(player, kris, 1000)
play(player, mrugesh, 1000)

Final results: {'p1': 991, 'p2': 5, 'tie': 4}
Player 1 win rate: 99.49799196787149%
Final results: {'p1': 443, 'p2': 315, 'tie': 242}
Player 1 win rate: 58.443271767810025%
Final results: {'p1': 644, 'p2': 209, 'tie': 147}
Player 1 win rate: 75.49824150058618%
Final results: {'p1': 818, 'p2': 166, 'tie': 16}
Player 1 win rate: 83.130081300813%


83.130081300813

## Play interactively against a bot:

In [ ]:
play(human, abbey, 20, verbose=True)

## Play against a bot that plays randomly

In [ ]:
play(human, random_player, 10)

## Tests

In [179]:
class UnitTests(unittest.TestCase):
    def test_player_vs_quincy(self):
        print("Testing game against quincy...")
        actual = play(player, quincy, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat quincy at least 60% of the time.')

    def test_player_vs_abbey(self):
        print("Testing game against abbey...")
        actual = play(player, abbey, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat abbey at least 60% of the time.')

    def test_player_vs_kris(self):
        print("Testing game against kris...")
        actual = play(player, kris, 1000) >= 60
        self.assertTrue(
            actual, 'Expected player to defeat kris at least 60% of the time.')

    def test_player_vs_mrugesh(self):
        print("Testing game against mrugesh...")
        actual = play(player, mrugesh, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat mrugesh at least 60% of the time.')

## Run the tests

In [186]:
unittest.main( argv=[''], verbosity=2, exit=False )

test_player_vs_abbey (__main__.UnitTests) ... ok
test_player_vs_kris (__main__.UnitTests) ... ok
test_player_vs_mrugesh (__main__.UnitTests) ... ok
test_player_vs_quincy (__main__.UnitTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.054s

OK


Testing game against abbey...
Final results: {'p1': 525, 'p2': 247, 'tie': 228}
Player 1 win rate: 68.00518134715026%
Testing game against kris...
Final results: {'p1': 994, 'p2': 2, 'tie': 4}
Player 1 win rate: 99.79919678714859%
Testing game against mrugesh...
Final results: {'p1': 840, 'p2': 159, 'tie': 1}
Player 1 win rate: 84.08408408408408%
Testing game against quincy...
Final results: {'p1': 997, 'p2': 2, 'tie': 1}
Player 1 win rate: 99.7997997997998%
